In [1]:
import re
import pandas as pd
import argparse
from openpyxl import load_workbook
pd.options.display.max_rows = None

In [9]:
# 開啟檔案
def open_brd_file(filepath):
    f = open(filepath)
    return f

#用空格拆分每一行
def splitRow(string):
    string = string.split()
    return string

#刪除|開頭
def deleteRow1(string):
    pattern = re.compile(r"\|")
    delete = pattern.findall(string)
    return delete

#刪除- - -開頭
def deleteRow2(string):
    pattern = re.compile(r"^\-$")
    delete = pattern.findall(string)
    return delete

#刪除R開頭
def deleteRow3(string):
    pattern = re.compile(r"^R")
    delete = pattern.findall(string)
    return delete

#刪多餘的raw
def deleteNullVIAROW(df):
    for i in range(df.shape[0]):
        if df.loc[i, "location"] == "VIA" and df.loc[i, "gap"] == 0.0:
            df.drop(i, axis=0, inplace=True)

# 存excel
def save_excel(write, df, sheetName):
#     df = pd.DataFrame(datalist)
    df.to_excel(write, sheet_name=f'{sheetName}', index=False)

    
#計算表層裡層
def cauculateMSSL(pathIDX):
    TTL = 0
    MS = 0
    SL = 0
    NAN = 0
    
    #MS SL
    for idx in pathIDX:
        TTL += dfSQSR.loc[idx, "gap"]
        ####################################################  前面的表層裡層         
        if re.findall("BOTTOM|TOP", dfSQSR.loc[idx, "layer"]): 
            MS += dfSQSR.loc[idx, "gap"]
        elif re.findall("L\d+$|IN\d+$", dfSQSR.loc[idx, "layer"]):
            SL += dfSQSR.loc[idx, "gap"]
        else:
            NAN += dfSQSR.loc[idx, "gap"]
        
        ##########################################################
    return TTL, MS, SL, NAN

#這一份txt total的path 數量
def branchPathNum(df):
    #扣掉net name/start_end/total length
    maxPathNum = int((df.shape[1] - 1) / 6)
    return maxPathNum

In [39]:
##########################RUN CODE######################################
#step1: 開啟檔案
filepath = "VRAM_0115c.rpt"
try:
    f = open_brd_file(filepath)
    print(f"step1: Open file. (File name: {filepath}) -DONE")
except:
    print(f"File is not exit, check file name.")
    
#step2: 刪除不必要行數
rawData = []
SQS = []
summary = []
netNameList = []

for i in f:
    i = i.replace("\n","")#去掉換行符號
    row = splitRow(i)

    if not deleteRow1(row[0]):
        if not deleteRow2(row[0]):
            rawData.append(row)
print(f"step2: Clear excess rows. -DONE")

#step3: 整理成需要的格式
for i in rawData:
#     if len(i) == 1: #netname那一行

    if len(i) < 3: #netname那一行 ######################### TODO這行應該要改
        data = {} #清空data
        netName = i[0]
        
        netNameList.append(netName) #建立netnamelist
    elif i[-1] == "0.000" or i[-1] == "0.00": #找到location

        data = {} #清空 
        try:
            location, length = i[0], float(i[-1])
            data.update({"net_name" : netName, "location" : location, "length": length, "layer": ""})
        except:
            print(netName," has error.")
        
        SQS.append(data)
    elif i[-1] == "mils": #找到TOTAL
        data = {} #清空
        ttlLength = float(i[3])
#         data.update({"net_name" : netName, "total_length" : ttlLength})
        data.update({"net_name" : netName })
        summary.append(data)
        
    else:
        data = {} #清空
        if not deleteRow3(i[0]):
            try:
                location, length, layer = i[0], float(i[-2]), i[-1]
                data.update({"net_name" : netName, "location" : location, "length": length, "layer": layer})
            except:
                print(netName, "has error.")

            SQS.append(data)

#存取df
dfsummary, dfSQS, dfSQSR = pd.DataFrame(summary), pd.DataFrame(SQS), pd.DataFrame(SQS)
print("step3: Transfer file format. -DONE")


#step4: 分析txt
#修改df
print("step4: Parsing txt file...")
print("       Calculating... - 25%")
#修改df
for i in netNameList:
    indexList = dfSQSR[dfSQSR["net_name"] == f"{i}"].index.tolist()
    
    # 填第一個元件的layer空格 
    # ex. TOP = TOP
    dfSQSR.loc[indexList[0],"layer"] = dfSQSR.loc[indexList[1],"layer"]
    
    #每個netname裡面找gap
    length = len(indexList)
    for index in indexList[::-1]:
        if length > 1:
            gap = dfSQSR.loc[index]["length"] - dfSQSR.loc[index-1]["length"]
            dfSQSR.loc[index,"gap"] = gap
        else:
            gap = 0
            dfSQSR.loc[index,"gap"] = gap
        length -= 1

print("       Calculating... - 50%")
#刪除多餘的VIA
deleteNullVIAROW(dfSQSR)


step1: Open file. (File name: VRAM_0115c.rpt) -DONE
step2: Clear excess rows. -DONE
step3: Transfer file format. -DONE
step4: Parsing txt file...
       Calculating... - 25%
       Calculating... - 50%


In [40]:
#在summary建立branch path & branch length
#####先把想要的資料塞到dfsummary裡面#######
for i in netNameList:
    indexList = dfSQSR[dfSQSR["net_name"] == f"{i}"].index.tolist()
    
    
    
    #有岔路沒岔路要分開跑 先把每條net的位子寫出來看有沒有VIA(T)
    locations = []
    for idx in indexList:
        locations.append(dfSQSR.loc[idx, "location"])

        


    #summary每一個netname的index
    netIndex = dfsummary[dfsummary["net_name"] == f"{i}"].index.tolist()

#如果有分岐
    if 'VIA(T)' in locations:
        #1
        #把起始點 & 終點 加入summary
        VIAT_IDX = locations.index('VIA(T)')
        length = len(indexList)
        U = re.compile("^U")
        UAfterVIAT = [item for i, item in enumerate(locations[VIAT_IDX:]) if re.search(U, item)]#找出VIAT後面的U
        U_IDX = [locations.index(i) for i in UAfterVIAT]
        
        for idx in U_IDX:#每個 U一條路            
            
            number = U_IDX.index(idx)
            pathIDX = indexList[0: VIAT_IDX + 1]
            
            if number > 0: #path2
                previousUIDX = U_IDX[number-1]
                pathIDX += indexList[previousUIDX + 1 : idx + 1]
                start_end = f"{dfSQSR.loc[pathIDX[0], 'location']}:{dfSQSR.loc[pathIDX[-1], 'location']}"
                dfsummary.loc[netIndex, f"path{number + 1}"] = start_end
                
                TTL, MS, SL , NAN = cauculateMSSL(pathIDX)
                dfsummary.loc[netIndex, f"path{number + 1}_TTL"] = TTL
                dfsummary.loc[netIndex, f"path{number + 1}_MS"] = start_end + "-MS"
                dfsummary.loc[netIndex, f"length{number + 1}_MS"] = MS
                dfsummary.loc[netIndex, f"path{number + 1}_SL"] = start_end + "-SL"
                dfsummary.loc[netIndex, f"length{number + 1}_SL"] = SL
                

                
            else:#第1個 #path1                
                pathIDX += indexList[VIAT_IDX +1 : idx + 1]
                start_end = f"{dfSQSR.loc[pathIDX[0], 'location']}:{dfSQSR.loc[pathIDX[-1], 'location']}"
                dfsummary.loc[netIndex, f"path{number + 1}"] = start_end
                
                #2 MS SL
                TTL, MS, SL , NAN = cauculateMSSL(pathIDX)
                dfsummary.loc[netIndex, f"path{number + 1}_TTL"] = TTL
                dfsummary.loc[netIndex, f"path{number + 1}_MS"] = start_end + "-MS"
                dfsummary.loc[netIndex, f"length{number + 1}_MS"] = MS
                dfsummary.loc[netIndex, f"path{number + 1}_SL"] = start_end + "-SL"
                dfsummary.loc[netIndex, f"length{number + 1}_SL"] = SL        
        
#只有一條路
    else:
        #1
        #把起始點 & 終點 加入summary
        start_end = f"{dfSQSR.loc[indexList[0], 'location']}:{dfSQSR.loc[indexList[-1], 'location']}"
        dfsummary.loc[netIndex, "path1"] = start_end
        
        #2 
        #MS SL
        TTL, MS, SL , NAN = cauculateMSSL(indexList)
        dfsummary.loc[netIndex, "path1_TTL"] = TTL
        dfsummary.loc[netIndex, "path1_MS"] = start_end + "-MS"
        dfsummary.loc[netIndex, "length1_MS"] = MS
        dfsummary.loc[netIndex, "path1_SL"] = start_end + "-SL"
        dfsummary.loc[netIndex, "length1_SL"] = SL
        
print("       Calculating... - 75%")

       Calculating... - 75%


In [41]:
#轉成final SQS資料
column1 = [] #第一列
column2 = [] #第二列
for row in range(dfsummary.shape[0]): #總共幾條 net = 幾行row    
    column1.append(dfsummary.loc[row,"net_name"])
    column2.append("")
    for num in range(branchPathNum(dfsummary)):

        column1.append(dfsummary.loc[row,f"path{num + 1}"])
        column2.append(dfsummary.loc[row,f"path{num + 1}_TTL"])
        
        column1.append(dfsummary.loc[row,f"path{num + 1}_MS"])
        column2.append(dfsummary.loc[row,f"length{num + 1}_MS"])
        
        column1.append(dfsummary.loc[row,f"path{num + 1}_SL"])
        column2.append(dfsummary.loc[row,f"length{num + 1}_SL"])

#轉成df       
final = {
    "SQS"    : column1,
    "length" : column2
}

#轉成df 順便去掉空值
dfFinal = pd.DataFrame(final).dropna(axis=0)

print("       Calculating... - 100%")
print("       Parse txt file. -DONE")
print("       Converted to sheet1. -DONE")

       Calculating... - 100%
       Parse txt file. -DONE
       Converted to sheet1. -DONE


In [43]:
# step5: 儲存檔案
filename = filepath.replace('.txt',"")
filename = filepath.replace('.rpt',"")
print(filename)
write = pd.ExcelWriter(f'{filename}.xlsx')
save_excel(write, dfFinal, "SQS")
write.save()

print(f"step6: Save data to {filename}.xlsx. -DONE")  

VRAM_0115c
step6: Save data to VRAM_0115c.xlsx. -DONE
